In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import preprocessing
from tpot import TPOTRegressor

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import math as math

In [2]:
df = pd.read_csv("C:/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/generated_datasets/clean_v2_01-02.csv")

In [3]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,distance_travelled
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,3,10,Tuesday,0,7,30,45,0.50
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,17,Saturday,1,0,30,45,1.69
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,9,16,Thursday,0,0,0,15,0.63
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,6,13,Wednesday,0,18,0,0,2.50
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,2,4,Wednesday,0,7,30,30,1.39


In [4]:
df['passenger_count'] = df['passenger_count'].astype('category')
df['year'] = df['year'].astype('category')
df['month'] = df['month'].astype('category')
df['day_of_month'] = df['day_of_month'].astype('category')
df['day_of_week'] = df['day_of_week'].astype('category')
df['weekend_weekday'] = df['weekend_weekday'].astype('category')
df['hour'] = df['hour'].astype('category')
df['time_30_min_bin'] = df['time_30_min_bin'].astype('category')
df['time_15_min_bin'] = df['time_15_min_bin'].astype('category')

In [5]:
df.dtypes

fare_amount            float64
pickup_longitude       float64
pickup_latitude        float64
dropoff_longitude      float64
dropoff_latitude       float64
passenger_count       category
year                  category
month                 category
day_of_month          category
day_of_week           category
weekend_weekday       category
hour                  category
time_30_min_bin       category
time_15_min_bin       category
distance_travelled     float64
dtype: object

In [6]:
df.columns

Index(['fare_amount', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'year',
       'month', 'day_of_month', 'day_of_week', 'weekend_weekday', 'hour',
       'time_30_min_bin', 'time_15_min_bin', 'distance_travelled'],
      dtype='object')

In [7]:
def encodeLabel(df ,inp):
    le = preprocessing.LabelEncoder()
    le.fit(df[inp])
    return le.transform(df[inp]) 

In [8]:
df['day_of_week'] = encodeLabel(df, 'day_of_week')

In [9]:
#Cleaning fare : not giving better results
#df = df[(df["fare_amount"] <= 100)]

In [10]:
#fare_round: not giving better results
#df['fare_roundoff'] = df['fare_amount'].apply(lambda x: int(round(x, 0)))

In [11]:
#distance_bin: not giving better results
#df['distance_travelled'] = df['distance_travelled'].apply(lambda x: int(math.ceil(x)))

In [12]:
df.shape

(10854270, 15)

In [13]:
df.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,distance_travelled
0,4.0,-73.994102,40.751041,-73.991257,40.754959,1,2015,3,10,5,0,7,30,45,0.50
1,6.1,-73.961686,40.713017,-73.952406,40.699525,1,2011,12,17,2,1,0,30,45,1.69
2,9.7,-73.978325,40.753808,-73.985068,40.756177,1,2010,9,16,4,0,0,0,15,0.63
3,6.9,-74.001737,40.719357,-73.992353,40.740713,1,2012,6,13,6,0,18,0,0,2.50
4,6.0,-73.966393,40.770531,-73.958328,40.781441,3,2015,2,4,6,0,7,30,30,1.39


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854270 entries, 0 to 10854269
Data columns (total 15 columns):
fare_amount           float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
passenger_count       category
year                  category
month                 category
day_of_month          category
day_of_week           int64
weekend_weekday       category
hour                  category
time_30_min_bin       category
time_15_min_bin       category
distance_travelled    float64
dtypes: category(8), float64(6), int64(1)
memory usage: 662.5 MB


In [15]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,day_of_week,distance_travelled
count,1.085427e+07,1.085427e+07,1.085427e+07,1.085427e+07,1.085427e+07,1.085427e+07,1.085427e+07
mean,1.132243e+01,-7.397522e+01,4.075097e+01,-7.397436e+01,4.075129e+01,2.996612e+00,3.353142e+00
std,9.707121e+00,4.172371e-02,3.368306e-02,4.042135e-02,3.656146e-02,2.016591e+00,4.241596e+00
min,-3.000000e+02,-7.597389e+01,3.901473e+01,-7.596753e+01,3.901156e+01,0.000000e+00,0.000000e+00
25%,6.000000e+00,-7.399228e+01,4.073655e+01,-7.399160e+01,4.073554e+01,1.000000e+00,1.250000e+00
50%,8.500000e+00,-7.398210e+01,4.075337e+01,-7.398061e+01,4.075384e+01,3.000000e+00,2.150000e+00
75%,1.250000e+01,-7.396830e+01,4.076754e+01,-7.396534e+01,4.076839e+01,5.000000e+00,3.910000e+00
max,2.010900e+03,-7.000039e+01,4.584237e+01,-7.000000e+01,4.593333e+01,6.000000e+00,6.163300e+02


In [16]:
df['passenger_count'] = encodeLabel(df, 'passenger_count')
df['year'] = encodeLabel(df, 'year')
df['month'] = encodeLabel(df, 'month')
df['day_of_month'] = encodeLabel(df, 'day_of_month')
df['day_of_week'] = encodeLabel(df, 'day_of_week')
df['weekend_weekday'] = encodeLabel(df, 'weekend_weekday')
df['hour'] = encodeLabel(df, 'hour')
df['time_30_min_bin'] = encodeLabel(df, 'time_30_min_bin')
df['time_15_min_bin'] = encodeLabel(df, 'time_15_min_bin')

###  Splitting data

In [17]:
from sklearn.cross_validation import train_test_split

C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
df_chopped = df.head(10000)

In [25]:
X = df_chopped[['pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'year',
       'month', 'day_of_month', 'day_of_week', 'weekend_weekday', 'hour',
       'time_30_min_bin', 'time_15_min_bin','distance_travelled']]

In [26]:
y = df_chopped[['fare_amount']]

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [28]:
X_train.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,year,month,day_of_month,day_of_week,weekend_weekday,hour,time_30_min_bin,time_15_min_bin,distance_travelled
9069,-74.002489,40.739794,-73.991965,40.753947,1,0,4,17,1,0,15,1,3,1.81
2603,-73.978336,40.783017,-73.978215,40.783857,1,2,2,19,3,1,10,1,2,0.09
7738,-73.967412,40.766295,-73.959985,40.771332,3,4,1,19,6,0,15,1,3,0.84
1579,-73.995445,40.725297,-74.003575,40.713927,1,5,5,23,5,0,14,1,2,1.44
5058,-73.983305,40.744389,-73.984521,40.727300,2,2,10,29,6,0,17,0,1,1.90


In [29]:
y_train.shape

(7000, 1)

In [ ]:
%%time

tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, cv = 3
                    ,max_eval_time_mins=1.5)

tpot.fit(X_train, y_train)

C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap_external.py:434: ImportWarning: Not importing directory C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\site-packages\google: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\shabbir.hasan\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap_external.py:434: ImportWarning: Not importing directory c:\users\shabbir.hasan\appdata\local\continuum\anaconda3\lib\site-packages\g

Generation 1 - Current best internal CV score: -14.817906366536059


In [ ]:
print(tpot.score(X_test, y_test))
tpot.export('tpot_iris_pipeline.py')

### Model building

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

In [ ]:
# Train the model using the training sets
regr.fit(X_train, y_train)

In [ ]:
# Make predictions using the testing set
test_label_pred = regr.predict(X_test)

In [ ]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, test_label_pred))

In [ ]:
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, test_label_pred))

### Predicting on kaggle dataset

In [ ]:
test_kaggle = pd.read_csv("/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/test.csv")

In [ ]:
#copying coe from cleaning file

In [ ]:

# creating year
test_kaggle['year'] = test_kaggle.pickup_datetime.apply(lambda x: x[0:4])

#creating month
test_kaggle['month'] = test_kaggle.pickup_datetime.apply(lambda x: x[5:7])

# #year_month
# test_kaggle['year_month'] = test_kaggle.pickup_datetime.apply(lambda x: x[0:7])

#Day_of_month
test_kaggle['day_of_month'] = test_kaggle.pickup_datetime.apply(lambda x: x[8:10])

#time
test_kaggle['time'] = test_kaggle.pickup_datetime.apply(lambda x: x[11:16])

#hour
test_kaggle['hour'] = test_kaggle.time.apply(lambda x: x[0:2])

#minute
test_kaggle['minute'] = test_kaggle.time.apply(lambda x: x[3:5])

#date
test_kaggle['date'] = test_kaggle.pickup_datetime.apply(lambda x: x[0:10])


#time_zone --just to make sure all are same
# tested they all are same, so no need of creating this field
#test_kaggle['time_zone'] = test_kaggle.pickup_datetime.apply(lambda x: x[-3:])


In [ ]:
# converting date column into date format
test_kaggle['date'] = pd.to_datetime(test_kaggle['date'])

# creating day of the week column
test_kaggle['day_of_week'] = test_kaggle['date'].dt.day_name()

In [ ]:
test_kaggle =  test_kaggle.drop('time', 1)

In [ ]:
test_kaggle =  test_kaggle.drop('date', 1)

In [ ]:
#storing key for submission
submission = pd.DataFrame()
submission.key = pd.DataFrame(test_kaggle.key)
storing_key = test_kaggle['key']
storing_key['key'] = test_kaggle['key']

#deleting key
test_kaggle =  test_kaggle.drop('key', 1)

In [ ]:
# checking split of rides according to day of the week
test_kaggle['day_of_week'].value_counts()

In [ ]:
# creating weekend/weekday column
# weekday = 0 & weekday = 1
test_kaggle['weekend_weekday'] = test_kaggle.day_of_week.apply(lambda x: '1' if x in ('Friday', 'Saturday', 'Sunday') else '0')

In [ ]:
#dropping datetime column
test_kaggle = test_kaggle.drop('pickup_datetime', 1)

In [ ]:
# converting data type of columns
test_kaggle['passenger_count'] = test_kaggle['passenger_count'].astype('category')
test_kaggle['year'] = test_kaggle['year'].astype('category')
test_kaggle['month'] = test_kaggle['month'].astype('category')
#test_kaggle['year_month'] = test_kaggle['year_month'].astype('category')
test_kaggle['day_of_week'] = test_kaggle['day_of_week'].astype('category')
test_kaggle['weekend_weekday'] = test_kaggle['weekend_weekday'].astype('category')

In [ ]:
test_kaggle.head()

In [ ]:
#Day_of_month
test_kaggle['time_30_min_bin'] = test_kaggle.minute.apply(lambda x: '30' if int(x) >= 30 else '0')
test_kaggle['time_15_min_bin'] = test_kaggle.minute.apply(lambda x: '0' if int(x) < 15 
                                              else ('15' if int(x) >= 15 and int(x) < 30 
                                                    else ('30' if int(x) >=30 and int(x) < 45 
                                                          else '45')))

In [ ]:
# changing data type
test_kaggle['time_15_min_bin'] = test_kaggle['time_15_min_bin'].astype('category')
test_kaggle['time_30_min_bin'] = test_kaggle['time_30_min_bin'].astype('category')
test_kaggle['hour'] = test_kaggle['hour'].astype('category')

In [ ]:
test_kaggle['day_of_month'] = test_kaggle['day_of_month'].astype('category')

In [ ]:
test_kaggle =  test_kaggle.drop('minute', 1)

In [ ]:
test_kaggle.dtypes

In [ ]:
#encoding day of week
test_kaggle['day_of_week'] = encodeLabel(test_kaggle, 'day_of_week')

In [ ]:
#calculating radial distance

#fun to calc dist
def radialDistance(lat1 , lon1, lat2, lon2):
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return round(d,2)

In [ ]:
%%time
test_kaggle['distance_travelled'] = test_kaggle.apply(lambda x: radialDistance(x['pickup_latitude'], x['pickup_longitude'], 
                                           x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

In [ ]:
#distance_bin: didnt work
#test_kaggle['distance_travelled'] = test_kaggle['distance_travelled'].apply(lambda x: int(math.ceil(x)))

In [ ]:
test_kaggle.head()

## predicting on kaggle final data

In [ ]:
# Make predictions using the testing set
test_kagle_label_pred = regr.predict(test_kaggle)

In [ ]:
#checking values
test_kagle_label_pred[1:5]

In [ ]:
#rounding of to two decimal
import numpy as np
test_kagle_label_pred = np.round(test_kagle_label_pred, decimals=2)

In [ ]:
#checking values
test_kagle_label_pred[1:5]

In [ ]:
#adding fare in submission
submission.fare_amount = pd.DataFrame(test_kagle_label_pred)

In [ ]:
col_names =  ['key', 'fare_amount']
submission  = pd.DataFrame(columns = col_names)


In [ ]:
submission.head()

In [ ]:
#i know i am stupid for doing this, but i am in a rush
test_kaggle_forkey = pd.read_csv("/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/test.csv")

In [ ]:
submission.key = test_kaggle_forkey['key']
submission.fare_amount = test_kagle_label_pred

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_v5.csv', index=False )

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
reg = RandomForestRegressor(n_estimators=1000, max_depth=3, random_state=0)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
test_label_pred_rf = reg.predict(X_test)

In [ ]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, test_label_pred_rf))

In [ ]:
test_label_pred_rf

In [ ]:
submission.key = test_kaggle_forkey['key']
submission.fare_amount = pd.DataFrame(test_label_pred_rf)

In [ ]:
submission['fare_amount'] = pd.DataFrame(test_label_pred_rf)

In [ ]:
submission['fare_amount'] = submission['fare_amount'].round(2)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission_v7_rf.csv', index=False )